In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
jtplot.style(figsize=(24, 9))

# TWAP. Time-Weighted Average Price

In [2]:
df = pd.read_csv('../../../data/EURUSD_Ticks_19.08.2024-19.08.2024.csv',
                 index_col=0, parse_dates=True)
df.head()

,Ask,Bid,AskVolume,BidVolume
Local time,,,,
2024-08-19 00:00:00.062000-02:00,1.10275,1.10254,1.8,0.45
2024-08-19 00:00:00.165000-02:00,1.10276,1.10254,1.8,0.45
2024-08-19 00:00:00.218000-02:00,1.10277,1.10251,1.8,0.35
2024-08-19 00:00:00.270000-02:00,1.10277,1.10257,0.9,0.90
2024-08-19 00:00:00.325000-02:00,1.10282,1.10250,0.9,0.45


In [3]:
df.shape

(62503, 4)

In [4]:
df['Spread'] = df['Ask'] - df['Bid']
df['MidPrice'] = (df['Ask'] + df['Bid']) / 2

df.head()

,Ask,Bid,AskVolume,BidVolume,Spread,MidPrice
Local time,,,,,,
2024-08-19 00:00:00.062000-02:00,1.10275,1.10254,1.8,0.45,0.00021,1.102645
2024-08-19 00:00:00.165000-02:00,1.10276,1.10254,1.8,0.45,0.00022,1.102650
2024-08-19 00:00:00.218000-02:00,1.10277,1.10251,1.8,0.35,0.00026,1.102640
2024-08-19 00:00:00.270000-02:00,1.10277,1.10257,0.9,0.90,0.00020,1.102670
2024-08-19 00:00:00.325000-02:00,1.10282,1.10250,0.9,0.45,0.00032,1.102660


In [5]:
df.tail()

,Ask,Bid,AskVolume,BidVolume,Spread,MidPrice
Local time,,,,,,
2024-08-19 23:59:51.424000-02:00,1.10848,1.10840,4.50,0.90,0.00008,1.108440
2024-08-19 23:59:51.527000-02:00,1.10850,1.10835,0.45,0.45,0.00015,1.108425
2024-08-19 23:59:51.579000-02:00,1.10853,1.10831,0.90,4.50,0.00022,1.108420
2024-08-19 23:59:51.783000-02:00,1.10853,1.10835,6.30,0.90,0.00018,1.108440
2024-08-19 23:59:59.970000-02:00,1.10851,1.10835,0.90,0.90,0.00016,1.108430


In [6]:
time_interval = 5 # in minutes
total_volume = 1 # 1 lot = 100_000

start_time = df.index.min()
end_time = start_time + pd.Timedelta(minutes=time_interval)
start_time, end_time

(Timestamp('2024-08-19 00:00:00.062000-0200', tz='pytz.FixedOffset(-120)'),
 Timestamp('2024-08-19 00:05:00.062000-0200', tz='pytz.FixedOffset(-120)'))

In [7]:
period_data = df.loc[(df.index>=start_time) & (df.index<=end_time)].copy()

In [14]:
num_slices = time_interval
volume_per_slice = total_volume / num_slices

num_slices, volume_per_slice

(5, 0.2)

In [13]:
slice_times = pd.date_range(
    start=start_time, 
    end=end_time, 
    periods=num_slices+1
)
slice_times

DatetimeIndex(['2024-08-19 00:00:00.062000-02:00',
               '2024-08-19 00:01:00.062000-02:00',
               '2024-08-19 00:02:00.062000-02:00',
               '2024-08-19 00:03:00.062000-02:00',
               '2024-08-19 00:04:00.062000-02:00',
               '2024-08-19 00:05:00.062000-02:00'],
              dtype='datetime64[ns, pytz.FixedOffset(-120)]', freq=None)

In [11]:
period_data.reset_index(inplace=True)
period_data.rename(columns={'Local time': 'timestamp'}, inplace=True)
period_data.head()

,timestamp,Ask,Bid,AskVolume,BidVolume,Spread,MidPrice
0,2024-08-19 00:00:00.062000-02:00,1.10275,1.10254,1.8,0.45,0.00021,1.102645
1,2024-08-19 00:00:00.165000-02:00,1.10276,1.10254,1.8,0.45,0.00022,1.102650
2,2024-08-19 00:00:00.218000-02:00,1.10277,1.10251,1.8,0.35,0.00026,1.102640
3,2024-08-19 00:00:00.270000-02:00,1.10277,1.10257,0.9,0.90,0.00020,1.102670
4,2024-08-19 00:00:00.325000-02:00,1.10282,1.10250,0.9,0.45,0.00032,1.102660


In [11]:
slice_data = period_data.iloc[
    (period_data['timestamp'] - slice_times[0]).abs().argsort()[:1]
].iloc[0]

slice_data

timestamp    2024-08-19 00:00:00.062000-02:00
Ask                                   1.10275
Bid                                   1.10254
AskVolume                                 1.8
BidVolume                                0.45
Spread                                0.00021
MidPrice                             1.102645
Name: 0, dtype: object

In [13]:
ask, bid = slice_data['Ask'], slice_data['Bid']
ask_vol, bid_vol = slice_data['AskVolume'], slice_data['BidVolume']

ask, bid, ask_vol, bid_vol

(1.10312, 1.10308, 2.25, 1.35)

In [39]:
deals_bid = []
volume_per_slice = 2.3
slice_data = period_data.iloc[(period_data['timestamp'] - slice_times[0]).abs().argsort()]

for index, row in slice_data.iterrows():
    if volume_per_slice > 0:
        if volume_per_slice <= row['BidVolume']:
            deals_bid.append({'price': row['Bid']})
            deals_bid.append({'volume': volume_per_slice})
            deals_bid.append({'price*vol': row['Bid']*volume_per_slice})
        else:
            deals_bid.append({'price': row['Bid']})
            deals_bid.append({'volume': row['BidVolume']})
            deals_bid.append({'price*vol': row['Bid']*row['BidVolume']})
            
        volume_per_slice -= row['BidVolume']
    else:
        print('Order was executed')
        break

deals_bid

Order was executed


[{'price': 1.10254},
 {'volume': 0.45},
 {'price*vol': 0.49614300000000006},
 {'price': 1.10254},
 {'volume': 0.45},
 {'price*vol': 0.49614300000000006},
 {'price': 1.10251},
 {'volume': 0.35},
 {'price*vol': 0.3858785},
 {'price': 1.10257},
 {'volume': 0.9},
 {'price*vol': 0.9923130000000001},
 {'price': 1.1025},
 {'volume': 0.1499999999999998},
 {'price*vol': 0.16537499999999977}]

In [15]:
deals_bid = []
deals_ask = []
for i in range(num_slices):
    slice_data = period_data.iloc[
        (period_data['timestamp'] - slice_times[i]).abs().argsort()
    ]

    for index, row in slice_data.iterrows():
        if volume_per_slice > 0:
            if volume_per_slice <= row['BidVolume']:
                deals_bid.append({'price': row['Bid']})
                deals_bid.append({'volume': volume_per_slice})
                deals_bid.append({'price*vol': row['Bid']*volume_per_slice})
            else:
                deals_bid.append({'price': row['Bid']})
                deals_bid.append({'volume': row['BidVolume']})
                deals_bid.append({'price*vol': row['Bid']*row['BidVolume']})

            volume_per_slice -= row['BidVolume']
        else:
            print('Order was executed')
            break



Order was executed
Order was executed
Order was executed
Order was executed
Order was executed


In [16]:
deals_bid

[{'price': 1.10254}, {'volume': 0.2}, {'price*vol': 0.22050800000000004}]